# Special Vector/Matrix/Tensor Operations, Attributes and Built-in Methods

In [1]:
import numpy as np
import scipy
import sympy
import tensorflow as tf
import torch
import random
import jax
import jax.numpy as jnp


from jax import jit, grad, vmap, jacobian
from scipy import linalg
from sympy.vector import CoordSys3D
from sympy.vector import Vector
from sympy.interactive.printing import init_printing
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from sympy import Array

# Vector Operations

## Numpy

In [2]:
v1 = np.array([5, 4, 2, 2])
v2 = np.array([1, 1, 3, 3])
v3 = np.array([[np.random.randint(0, 100) for _ in range(3)]]) 
v4 = np.array([[np.random.randint(-50, 0) for _ in range(3)]]) # vertical vector

In [3]:
# Dot Product of vectors
print(f"Dot product v1 and v2 = {v1.dot(v2)}")

Dot product v1 and v2 = 21


In [4]:
#Cross Product
print(f"Cross Product v3-v4: {np.cross(v3, v4)}")

Cross Product v3-v4: [[-1520  1519   905]]


In [5]:
#Magnitude/Length of the vector
print(f"Magnitude of the vector v4: {np.linalg.norm(v4)}")

Magnitude of the vector v4: 55.57877292636101


## Sympy

In [6]:
N = CoordSys3D('N')

In [7]:
# Define a vector with basevectors
v1 = 2 * N.i + 5 * N.j
v2 = -3 * N.i - 2 * N.j

w1 = 4 * N.i + 2 * N.j - 3 * N.k
w2 = -5 * N.i - 2 * N.j + 9 * N.k

In [8]:
# Dot Product of vectors
print(f"Dot Product v1 and v2: {v1.dot(v2)}")

Dot Product v1 and v2: -16


In [9]:
# Cross Product of vectors
print(f"Cross Product of w1 and w2: {w1.cross(w2)}")

Cross Product of w1 and w2: 12*N.i + (-21)*N.j + 2*N.k


In [10]:
# Magnitude and normalization to have a 1 magnitude
print(f"Magnitude/Length of a vector: {v1.magnitude()}")
print(f"Normalization: {v1.normalize()}")

Magnitude/Length of a vector: sqrt(29)
Normalization: (2*sqrt(29)/29)*N.i + (5*sqrt(29)/29)*N.j


## JAX

In [11]:
key = jax.random.PRNGKey(42)

In [12]:
v1 = jnp.array([5, 4, 2, 2])
v2 = jnp.array([1, 1, 3, 3])
v3 = jax.random.randint(key, shape=(2, 3), minval=5, maxval=50)
v4 = jax.random.randint(key, shape=(2, 3), minval=2, maxval=25)

In [13]:
# Dot Product of vectors
print(f"Dot product v1 and v2 = {v1.dot(v2)}")

Dot product v1 and v2 = 21


In [14]:
#Cross Product
print(f"Cross Product v3-v4: \n{jnp.cross(v3, v4)}")

Cross Product v3-v4: 
[[ -47 -189  234]
 [ 318 -479  379]]


In [15]:
#Magnitude/Length of the vector
print(f"Magnitude of the vector v1: {jnp.linalg.norm(v1)}")

Magnitude of the vector v1: 7.0


###  Just-in-time compilation

In [16]:
def vector_refactor(x, alpha=1.25):  
    return jnp.sqrt(2.5 * jnp.where(x < 5, x, x ** 2 - alpha))

In [17]:
key = jax.random.PRNGKey(42)

In [18]:
x = jax.random.normal(key, (1_000_000,))
%timeit vector_refactor(x).block_until_ready()

6.08 ms ± 434 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
vector_refactor_jit = jit(vector_refactor)
_ = vector_refactor(x)
%timeit vector_refactor(x).block_until_ready()

6.13 ms ± 439 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
x.devices()

{CpuDevice(id=0)}

### Taking derivatives

In [21]:
def sum_sigmoid_values(x):
  return jnp.sum(jax.nn.sigmoid(x) ** 3)

In [22]:
x = jnp.array([1.0, 2.0, 3.0])

In [23]:
sum_sigmoid_values(x)

Array(1.9384005, dtype=float32)

In [24]:
derivative_fn = grad(sum_sigmoid_values)
print(derivative_fn(x))

[0.31523573 0.24436323 0.12297955]


In [25]:
jac_sum_sigmoid = jacobian(sum_sigmoid_values)
print(jac_sum_sigmoid(x))

[0.31523573 0.24436323 0.12297955]


In [26]:
# JIT can be used as well
jit_jac_sum_sigmoid = jit(jacobian(sum_sigmoid_values))
print(jit_jac_sum_sigmoid(x))

[0.31523573 0.24436323 0.12297955]


# Matrix Operations

## Numpy

In [27]:
A_array = np.random.randint(5, 30, size=(3, 3))
A_matrix = np.matrix([[2, 5, 6], [5, 7, 8], [12, 15, 22]], dtype="int")
B = np.random.randint(5, 30, size=(2, 3))
C = np.random.randint(5, 30, size=(2, 3))
D = np.random.randint(5, 30, size=(3, 2))

In [28]:
# Matrix multiplication element-wise at same shape
print(f"Matrix multiplication element-wise at same shape matrices: B * C = \n{B * C}")

Matrix multiplication element-wise at same shape matrices: B * C = 
[[390 812 189]
 [513 120 224]]


In [29]:
# Matrix multiplication
C @ D
#or
np.matmul(C, D)

array([[ 884, 1297],
       [ 616,  933]])

In [30]:
# Transpose of a Matrix
C.T

array([[26, 19],
       [29,  6],
       [ 9, 16]])

In [31]:
# Inverse of a Matrix
try:
    A_array.I
except:
    print("Numpy Array doesn't have inverse attribute")
    print("Trying Linalg\n")
    print(linalg.inv(A_matrix))

Numpy Array doesn't have inverse attribute
Trying Linalg

[[-0.60714286  0.35714286  0.03571429]
 [ 0.25        0.5        -0.25      ]
 [ 0.16071429 -0.53571429  0.19642857]]


In [32]:
# Inverse of a Matrix (method 2)
A_matrix.I

matrix([[-0.60714286,  0.35714286,  0.03571429],
        [ 0.25      ,  0.5       , -0.25      ],
        [ 0.16071429, -0.53571429,  0.19642857]])

In [33]:
# Calculate Determinant of a Matrix 
print(f"Calculating Determinant of matrix A: {linalg.det(A_matrix)}")

Calculating Determinant of matrix A: -56.0


In [34]:
# Eigenvalues and eigenvectors
eigenvalues, eigenvectors = linalg.eig(A_matrix)

In [35]:
eigenvalues

array([31.10286906+0.j, -1.3942385 +0.j,  1.29136944+0.j])

In [36]:
eigenvectors

array([[-0.24653765, -0.91712674, -0.17621159],
       [-0.35097351,  0.25182041, -0.74586549],
       [-0.90334754,  0.30897414,  0.64236606]])

### Useful built-in methods in Numpy

In [37]:
C

array([[26, 29,  9],
       [19,  6, 16]])

In [38]:
# Returning max value index in a given axis
C.argmax(axis=None) #axis=0 col-wise, axis=1 row-wise

1

In [39]:
# Returning max value based on argmax index
C.flatten()[C.argmax()]

29

In [40]:
# Returns the indices that would sort this array
C.argsort(axis=None) #axis=0 col-wise, axis=1 row-wise

array([4, 2, 5, 3, 0, 1], dtype=int64)

In [41]:
# Return selected slices of this array along given axis
np.choose([1, 0, 1], C)

array([19, 29, 16])

In [42]:
# Useful 1D iterator on flattened matrix
for i in C.flat: 
    print(i)

26
29
9
19
6
16


In [43]:
# Return an array whose values are limited to given min-max
C.clip(22, 30)

array([[26, 29, 22],
       [22, 22, 22]])

In [44]:
# Returning Max, Mean, Min, Sum 
print(f"C matrix max: {C.max(axis=None)}")
print(f"C matrix mean: {C.mean(axis=None)}")
print(f"C matrix min: {C.min(axis=None)}")
print(f"C matrix sum: {C.sum(axis=None)}")
print(f"C matrix std: {C.std(axis=None)}")

C matrix max: 29
C matrix mean: 17.5
C matrix min: 6
C matrix sum: 105
C matrix std: 8.301606270274847


In [45]:
C.cumsum(axis=None) # cum sum along axis 1

array([ 26,  55,  64,  83,  89, 105])

In [46]:
# Return shape of matrix
C.shape

(2, 3)

In [47]:
# Resize array in place
C.resize(1, 6)

In [48]:
# Return the matrix as a (possibly nested) list
C.tolist()

[[26, 29, 9, 19, 6, 16]]

In [49]:
# Return diagonal of a matrix (easier to visualise at a square matrix)
A_matrix.diagonal()

matrix([[ 2,  7, 22]])

## Sympy

In [50]:
# Define Matrices
A = Matrix([[12, 25, 3], [24, 100, 1]])
B =  Matrix(3, 3, np.random.randint(0, 15, 9))
C = Matrix(4, 4, lambda i,j: 2 + (i-j))
D = Matrix(3, 3, np.random.randint(15, 50, 9))

In [51]:
# Matrix Multiplication (no element-wise at Sympy)
A @ B
# or A * B

Matrix([
[ 506,  425,  96],
[1672, 1525, 132]])

In [52]:
# Take transpose of a matrix
A.T

Matrix([
[12,  24],
[25, 100],
[ 3,   1]])

In [53]:
# Compute determinant of a matrix
D.det()

29394

In [54]:
D.inv()

Matrix([
[    -2/69,      7/207,     -1/207],
[-137/9798, -383/29394,  604/14697],
[ 251/4899,  -85/14697, -461/14697]])

In [55]:
# Eigenvalues of a matrix with algebraic-multiplicity
C.eigenvals()

{4 - 2*I: 1, 4 + 2*I: 1, 0: 2}

In [56]:
# Eigenvectors of a matrix
C.eigenvects()

[(0,
  2,
  [Matrix([
   [ 1],
   [-2],
   [ 1],
   [ 0]]),
   Matrix([
   [ 2],
   [-3],
   [ 0],
   [ 1]])]),
 (4 - 2*I,
  1,
  [Matrix([
   [-2/13 - 3*I/13],
   [ 3/13 - 2*I/13],
   [   8/13 - I/13],
   [             1]])]),
 (4 + 2*I,
  1,
  [Matrix([
   [-2/13 + 3*I/13],
   [ 3/13 + 2*I/13],
   [   8/13 + I/13],
   [             1]])])]

### Basic Operations

In [57]:
D

Matrix([
[19, 32, 39],
[49, 34, 37],
[22, 46, 25]])

In [58]:
# Printing out matrix shape
D.shape

(3, 3)

In [59]:
# Accessing first column
D.col(0)

Matrix([
[19],
[49],
[22]])

In [60]:
# Accessing Last row
D.row(-1)

Matrix([[22, 46, 25]])

In [61]:
# Deleting row/column
D.col_del(0)
D.row_del(-1)

In [62]:
D

Matrix([
[32, 39],
[34, 37]])

## JAX

In [63]:
# Define Matrices
A = jnp.array([[12, 25, 3], [24, 100, 1]])
B =  jnp.array([[12, 25, 3], [24, 100, 1], [56, 120, 9]])
C = jnp.array([[3.5, 2.21, 1.12], [1.2, 2, 3.3]])

In [64]:
# Matrix Element-wise multiplication
A * C

Array([[ 42.       ,  55.25     ,   3.3600001],
       [ 28.800001 , 200.       ,   3.3      ]], dtype=float32)

In [65]:
# Matrix Multiplication
A @ B

Array([[  912,  3160,    88],
       [ 2744, 10720,   181]], dtype=int32)

In [66]:
# Transpose
A.T

Array([[ 12,  24],
       [ 25, 100],
       [  3,   1]], dtype=int32)

In [67]:
# Determinant
jnp.linalg.det(B)

Array(-2800., dtype=float32)

In [68]:
# Eigenvalues and Eigenvectors
eigvals, eigvecs = jnp.linalg.eig(B)
print("Eigenvalues:\n", eigvals)
print("Eigenvectors:\n", eigvecs)

Eigenvalues:
 [108.608894 +0.j  -1.8147824+0.j  14.2058525+0.j]
Eigenvectors:
 [[-0.17464584+0.j -0.32766196+0.j  0.28130454+0.j]
 [-0.579357  +0.j  0.06798168+0.j -0.0898279 +0.j]
 [-0.7961434 +0.j  0.9423461 +0.j  0.955405  +0.j]]


In [69]:
# Matrix Power
jnp.linalg.matrix_power(B, 2)

Array([[  912.,  3160.,    88.],
       [ 2744., 10720.,   181.],
       [ 4056., 14480.,   369.]], dtype=float32)

###  Just-in-time compilation works as well

### Taking Jacobian

In [70]:
D = jax.random.uniform(key, shape=(3, 3))

In [71]:
def sum_rows_times_sigmoid_values(x):
    x = jnp.atleast_2d(x)
    
    sigmoid_x = jax.nn.sigmoid(x)
    row_sums = jnp.sum(x, axis=1, keepdims=True)
    result = sigmoid_x * row_sums
    
    return result

In [72]:
sum_rows_times_sigmoid_values(D)

Array([[0.7605113 , 0.67269653, 0.6359098 ],
       [1.2769414 , 1.2147048 , 0.9878393 ],
       [0.71974444, 0.6918695 , 0.80271053]], dtype=float32)

In [73]:
jac_sum_sigmoid = jacobian(sum_rows_times_sigmoid_values)
print(jac_sum_sigmoid(x))

[[[1.9107301  0.7310586  0.7310586 ]
  [0.880797   1.5107588  0.880797  ]
  [0.95257413 0.95257413 1.223634  ]]]


# Tensor Operations

## TensorFlow

In [74]:
# Define Tensor
tensor_1 = tf.constant([
  [[0, 1, 2],
   [5, 6, 7]],
    
  [[10, 11, 12],
   [15, 16, 17]],
    
  [[20, 21, 22],
   [25, 26, 27]],])

tensor_2 = tf.random.uniform([3, 2, 3])

In [75]:
tensor_2

<tf.Tensor: shape=(3, 2, 3), dtype=float32, numpy=
array([[[0.90020275, 0.06798148, 0.6896994 ],
        [0.51883876, 0.8579979 , 0.48854697]],

       [[0.920483  , 0.87047386, 0.08240378],
        [0.16821802, 0.63025284, 0.7074007 ]],

       [[0.25317442, 0.8379487 , 0.09093046],
        [0.94929206, 0.6417327 , 0.9382738 ]]], dtype=float32)>

In [76]:
print("Shape:", tensor_2.shape)

Shape: (3, 2, 3)


In [77]:
# Transpose of a Tensor
print("Transpose:\n", tf.transpose(tensor_2))

Transpose:
 tf.Tensor(
[[[0.90020275 0.920483   0.25317442]
  [0.51883876 0.16821802 0.94929206]]

 [[0.06798148 0.87047386 0.8379487 ]
  [0.8579979  0.63025284 0.6417327 ]]

 [[0.6896994  0.08240378 0.09093046]
  [0.48854697 0.7074007  0.9382738 ]]], shape=(3, 2, 3), dtype=float32)


In [78]:
# Sum, Min, Max, Mean
print("Sum:", tf.reduce_sum(tensor_2, axis=None, keepdims=False))
print()
print("Min:", tf.reduce_min(tensor_2, axis=1, keepdims=True))
print()
print("Max:", tf.reduce_max(tensor_2, axis=0, keepdims=False))
print()
print("Mean:", tf.reduce_mean(tensor_2))

Sum: tf.Tensor(10.613851, shape=(), dtype=float32)

Min: tf.Tensor(
[[[0.51883876 0.06798148 0.48854697]]

 [[0.16821802 0.63025284 0.08240378]]

 [[0.25317442 0.6417327  0.09093046]]], shape=(3, 1, 3), dtype=float32)

Max: tf.Tensor(
[[0.920483   0.87047386 0.6896994 ]
 [0.94929206 0.8579979  0.9382738 ]], shape=(2, 3), dtype=float32)

Mean: tf.Tensor(0.5896584, shape=(), dtype=float32)


## PyTorch

In [79]:
tensor_1 = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
tensor_2 = torch.tensor([[[25, 2], [62, 13]], [[24, 42], [4, 1]]])
tensor_random = torch.rand([2, 5, 6])

In [80]:
tensor_1

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

In [81]:
tensor_1.shape

torch.Size([2, 2, 2])

In [82]:
# Transpose of a Tensor
print("Transpose:\n", tf.transpose(tensor_2))

Transpose:
 tf.Tensor(
[[[25 24]
  [62  4]]

 [[ 2 42]
  [13  1]]], shape=(2, 2, 2), dtype=int64)


In [83]:
tensor_2

tensor([[[25,  2],
         [62, 13]],

        [[24, 42],
         [ 4,  1]]])

In [84]:
# Sum, Min, Max, Mean
print("Sum:", torch.sum(tensor_2, axis=None, keepdims=False))
print()
print("Min:", torch.min(tensor_2, axis=1, keepdims=True).values) #if only values
print()
print("Max:", torch.max(tensor_2, axis=0, keepdims=False)) #return max values and indexes where the value(s) were found
print()
print("Mean:", torch.mean(tensor_2, dtype=torch.float)) #cast to float before mean, since float result!

Sum: tensor(173)

Min: tensor([[[25,  2]],

        [[ 4,  1]]])

Max: torch.return_types.max(
values=tensor([[25, 42],
        [62, 13]]),
indices=tensor([[0, 1],
        [0, 0]]))

Mean: tensor(21.6250)


In [85]:
# Matrix Multiplication
tensor_1 @ tensor_2
# OR
torch.matmul(tensor_1, tensor_2)

tensor([[[149,  28],
         [323,  58]],

        [[144, 216],
         [200, 302]]])

## JAX

In [86]:
key = jax.random.PRNGKey(42)

In [87]:
# Define Tensor
tensor_1 = jnp.array([[[25, 12, 23], [3, 4, 5]], [[6, 7, 8], [213, 1, 2]]])
tensor_2 = jnp.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
tensor_3 = jax.random.randint(key, shape=(2, 2, 3), minval=0, maxval=15)

In [88]:
tensor_1.shape

(2, 2, 3)

In [89]:
jnp.reshape(tensor_1, (2, 6))

Array([[ 25,  12,  23,   3,   4,   5],
       [  6,   7,   8, 213,   1,   2]], dtype=int32)

In [90]:
# Concatenate Tensors along an axis
jnp.concatenate((tensor_1, tensor_3), axis=0)

Array([[[ 25,  12,  23],
        [  3,   4,   5]],

       [[  6,   7,   8],
        [213,   1,   2]],

       [[ 12,   8,   9],
        [  7,   1,   5]],

       [[ 11,  10,   4],
        [  9,   1,   2]]], dtype=int32)

In [91]:
# Stacking tensor_1 first col and tensor_3 third col
jnp.stack((tensor_1[:, :, 0].flatten(), tensor_3[:, :, 2].flatten()), axis=1)

Array([[ 25,   9],
       [  3,   5],
       [  6,   4],
       [213,   2]], dtype=int32)

In [92]:
# Splitting along an axis into sub-tensors
jnp.split(tensor_1, 3, axis=2)

[Array([[[ 25],
         [  3]],
 
        [[  6],
         [213]]], dtype=int32),
 Array([[[12],
         [ 4]],
 
        [[ 7],
         [ 1]]], dtype=int32),
 Array([[[23],
         [ 5]],
 
        [[ 8],
         [ 2]]], dtype=int32)]

In [93]:
# Transpose Tensor
jnp.transpose(tensor_1)

Array([[[ 25,   6],
        [  3, 213]],

       [[ 12,   7],
        [  4,   1]],

       [[ 23,   8],
        [  5,   2]]], dtype=int32)

In [94]:
tensor_1

Array([[[ 25,  12,  23],
        [  3,   4,   5]],

       [[  6,   7,   8],
        [213,   1,   2]]], dtype=int32)

In [95]:
# Sum, Min, Max, Mean
print(jnp.sum(tensor_1, axis=1))
print()
print(jnp.min(tensor_1, axis=0))
print()
print(jnp.max(tensor_1, axis=2))
print()
print(jnp.mean(tensor_1, axis=1))

[[ 28  16  28]
 [219   8  10]]

[[6 7 8]
 [3 1 2]]

[[ 25   5]
 [  8 213]]

[[ 14.    8.   14. ]
 [109.5   4.    5. ]]


###  Just-in-time compilation works as well

### Taking Jacobian

In [96]:
D = jax.random.uniform(key, shape=(2, 3, 3))

In [97]:
def sum_rows_times_sigmoid_values(x):
    x = jnp.atleast_2d(x)
    
    sigmoid_x = jax.nn.sigmoid(x)
    row_sums = jnp.sum(x, axis=0, keepdims=True)
    result = sigmoid_x * row_sums
    
    return result

In [98]:
sum_rows_times_sigmoid_values(D)

Array([[[0.47287682, 0.3414411 , 0.6920294 ],
        [0.74847937, 0.4612311 , 0.9051011 ],
        [0.42102578, 1.143624  , 0.3751168 ]],

       [[0.53499585, 0.4087908 , 0.74047166],
        [0.8687725 , 0.6095541 , 1.0656195 ],
        [0.38845807, 1.0286559 , 0.4915805 ]]], dtype=float32)

In [99]:
jac_sum_sigmoid = jacobian(sum_rows_times_sigmoid_values)
print(jac_sum_sigmoid(x))

[[[0.92767054 0.         0.        ]
  [0.         1.0907843  0.        ]
  [0.         0.         1.0881041 ]]]
